In [ ]:
import torch
import torch.optim as optim

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%aimport listnet

In [ ]:
n_train = 500
n_valid = 100
D = 50
n_epochs = 10
batch_size = 16

X_train, X_valid, ys_train, ys_valid = listnet.make_dataset(n_train, n_valid, D)

In [ ]:
X_train.shape

In [ ]:
ys_train.shape

In [ ]:
# モデルを宣言
net = listnet.Net(D)
optimizer = optim.Adam(net.parameters())

In [ ]:
for epoch in range(n_epochs):
	# ランダムに学習データを並べ替える
	idx = torch.randperm(n_train)
	X_train = X_train[idx]
	ys_train = ys_train[idx]

	cur_batch = 0
	for it in range(n_train // batch_size):
		# バッチをクエリに対応させる
		batch_X = X_train[cur_batch: cur_batch + batch_size]
		batch_ys = ys_train[cur_batch: cur_batch + batch_size]
		cur_batch += batch_size

		optimizer.zero_grad()
		if len(batch_X) > 0:
			# 現在のモデルの重みで予測を行う
			batch_pred = net(batch_X)

			# 本バッチの予測と正解を比較して損失を計算
			batch_loss = listnet.listnet_loss(batch_ys, batch_pred)

			# requires_grad=True とした変数に対して勾配を計算
			batch_loss.backward(retain_graph=True)

			# モデルの重みを更新する
			optimizer.step()

	with torch.no_grad():		# メモリ消費を抑えるために勾配計算を行わない
		valid_pred = net(X_valid)
		valid_swapped = listnet.swapped_pairs(valid_pred, ys_valid)
		ndcg_score = listnet.ndcg(ys_valid, valid_pred).item()		# Pythonの型に合わせるためにitem()を使う
		print('epoch: {}, swapped: {}, ndcg: {}'.format(epoch, valid_swapped, ndcg_score))